In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import geemap
import ee


In [2]:
ee.Authenticate()
ee.Initialize(project='twoearthengineproject')

In [3]:
physio_koshi = gpd.read_file(r"G:\learn\python_works\Work_projects\Koshi_chetan\chetan_cas\works\data\gis\koshi_basin_nepal\koshi_nepal_basin.shp")
whole_koshi_basin = physio_koshi.dissolve()
ee_physio_koshi = ee.FeatureCollection(geemap.geopandas_to_ee(whole_koshi_basin))

In [4]:
# Center map on basin
Map = geemap.Map()
Map.centerObject(ee_physio_koshi, 8)

# Add basin boundary with properties
Map.addLayer(
    ee_physio_koshi,
    {"color": "red"},
    "Koshi Basin Boundary"
)

In [5]:
# Get Sentinel-2 image collection
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(ee_physio_koshi) \
    .filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()

# Visualize in RGB
rgb_vis = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000
}

Map.addLayer(s2.clip(ee_physio_koshi), rgb_vis, "Sentinel-2 RGB")

In [6]:
# Get Sentinel-2 image collection
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(ee_physio_koshi) \
    .filterDate('2024-01-01', '2024-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()

# Visualize in RGB
rgb_vis = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000
}

Map.addLayer(s2.clip(ee_physio_koshi), rgb_vis, "Sentinel-2 RGB")

In [7]:
# NDWI calculation
ndwi = s2.normalizedDifference(["B3", "B8"]).rename("NDWI")

# Threshold NDWI to get water mask
water_mask = ndwi.gt(0.3)  # Adjust threshold for better results

Map.addLayer(ndwi.clip(ee_physio_koshi), {"min": 0, "max": 1, "palette": ["white", "blue"]}, "NDWI")
Map.addLayer(water_mask.updateMask(water_mask).clip(ee_physio_koshi), {"palette": ["#0000FF"]}, "Water Mask")


In [8]:
Map

Map(center=[27.459741378291714, 86.82980172772386], controls=(WidgetControl(options=['position', 'transparent_…